Conn with database

In [1]:
import sqlite3

conn = sqlite3.connect("gis_ele_neplan_noduri.gpkg")
c = conn.cursor()


Upload data to a pandas dataframe

In [2]:
import pandas as pd

df = pd.read_sql_query("SELECT * from v_neplan_nodes_all", conn)

A new dataframe with uniqueness on the node1 column

In [3]:
df_cleaned = df.drop_duplicates(subset = ['node1'], keep = 'first')
df_cleaned = df_cleaned.drop(columns=['geometry'])


For each row of the df_clean & set the attribute value according to the conditions

In [4]:
for index, row in df_cleaned.iterrows():
    node = row['node1']
    xdm_fno = row['xdm_fno']
    if xdm_fno in [313, 310, 304]:
        df_cleaned.loc[index, 'attribute'] = 'HV'
    elif xdm_fno in [314, 311, 305]:
        df_cleaned.loc[index, 'attribute'] = 'MV'
    else:
        df_cleaned.loc[index, 'attribute'] = ''

Create a new table v_neplane....cleaned in database

In [5]:
c.execute('''CREATE TABLE IF NOT EXISTS v_neplan_nodes_cleaned
            (fid INTEGER PRIMARY KEY,
            xdm_fid INTEGER,
            xdm_fno INTEGER,
            node1 TEXT,
            attribute TEXT)''')

df_cleaned.to_sql('v_neplan_nodes_cleaned', conn, if_exists='replace', index=False)

15417

Counter

In [6]:
from collections import Counter

counter = Counter()
for index, row in df_cleaned.iterrows():
    counter[row['node1']] += 1
